In [ ]:
import os
import glob
import pandas as pd
pd.set_option('display.max_colwidth', None)  
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['animation.html'] = 'jshtml'
import seaborn as sns
import datetime
import csv

import pydicom
import torch

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader



In [ ]:
root_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'

df = pd.read_csv(root_dir+'train_labels.csv')
bad_image = [109, 123, 709]
df = df[~df['BraTS21ID'].isin(bad_image)].reset_index(drop=True)
print(len(df['MGMT_value']==1))
print(len(df['MGMT_value']==0))

In [ ]:
# Load patiennt path
# zfill example, id=5, output = 00005
patient_path=[]
for i in range(df.shape[0]):
    id = df.iloc[i]["BraTS21ID"]
    patient_path.append(os.path.join("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/",str(id).zfill(5)))

In [ ]:

directory ={'FLAIR':[],'T1w':[],'T1wCE':[],'T2w':[]}
for path in patient_path:
    directory['FLAIR'].append(os.path.join(path, 'FLAIR'))
    directory['T1w'].append(os.path.join(path, 'T1w'))
    directory['T1wCE'].append(os.path.join(path, 'T1wCE'))
    directory['T2w'].append(os.path.join(path, 'T2w'))
    
directory = pd.DataFrame(directory)

In [ ]:
def get_stacked_images(images_slices):
    images = []
    for image in images_slices:
        if(np.max(image)!=0):
            images.append(image) 
    stacked_images = np.stack(images,0)
    return stacked_images
    
def load_image(file):
    data = pydicom.read_file(file)
    data = data.pixel_array
    return data

In [ ]:
import torch.nn.functional as F
import time
MAX_PIXEL_VAL = 255
MEAN = 58.09
STD = 49.73


def preprocess_data(path, transform=None):
#     start = time.time()
    images_path = sorted(glob.glob(os.path.join(path,"*")),key=lambda x: int(x[:-4].split("-")[-1]))
    images_slices = [load_image(image_path) for image_path in images_path]
    series = get_stacked_images(images_slices).astype(np.float32)
#     print('time:',time.time()-start)
    series = torch.tensor(np.stack((series,)*3, axis=1))
    resized = F.interpolate(series,(256))
    resized = resized.permute(0,1,3,2)
    resized = F.interpolate(resized,(256))
    series = resized.permute(0,1,3,2)
#     print(series.shape)

    if transform is not None:
        for i, slice in enumerate(series.split(1)):
            series[i] = transform(slice.squeeze())

    series = (series - series.min()) / (series.max() - series.min()) * MAX_PIXEL_VAL
    series = (series - MEAN) / STD
#     print('time:',time.time()-start)
    return series

In [ ]:
class customDataset(Dataset):
    def __init__(self, dataset_dir, labels, transform=None, device=None):
        self.paths = dataset_dir
        self.labels = labels
        self.transform = transform
        self.device = device
        if self.device is None:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):

        path = self.paths[idx]
        series = preprocess_data(path, self.transform)
        labels = torch.tensor(self.labels[idx].astype(np.float32))
        return (series, labels)

In [ ]:
def split_dataset(dataset_dir,labels):
    
    train_size = int(len(dataset_dir)*0.7)
    valid_size = len(dataset_dir)-train_size
    train_dataset = dataset_dir[0:train_size]
    train_labels = labels[0:train_size]
    valid_dataset = dataset_dir[-valid_size:].reset_index(drop=True)
    valid_labels = labels[-valid_size:].reset_index(drop=True)
    
    return train_dataset,train_labels,valid_dataset,valid_labels

def make_dataset(directory,mri_type, dataset_type, device=None):
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dataset = directory[mri_type]
    labels = df['MGMT_value']
    
    train_dataset,train_labels,valid_dataset,valid_labels = split_dataset(dataset,labels)
    
    if dataset_type == 'train':
        dataset_dir = train_dataset
        labels = train_labels
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(),
            transforms.RandomAffine(25, translate=(0.1, 0.1)),
            transforms.ToTensor()
        ])
    elif dataset_type == 'valid':
        dataset_dir = valid_dataset
        labels = valid_labels
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor()
        ])
    else:
        raise ValueError('Dataset needs to be train or valid.')

    dataset = customDataset(dataset_dir, labels, transform=transform, device=device)

    return dataset


In [ ]:
def make_data_loader(directory,mri_type,dataset_type,device=None,shuffle=False):
    
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dataset = make_dataset(directory,mri_type,dataset_type,device)
    data_loader = DataLoader(dataset, batch_size=1, shuffle=shuffle)

    return data_loader

# Create Model

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

class MRNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.alexnet = torch.load('../input/alexnet/alexnet.pth').features
        self.fc = nn.Linear(256, 1)

        self.avg_pool = nn.AvgPool2d(kernel_size=7, stride=None, padding=0)
        self.dropout = nn.Dropout(p=0.5)

    @property
    def features(self):
        return self.alexnet

    @property
    def classifier(self):
        return self.fc

    def forward(self, batch):
        batch_out = torch.tensor([]).to(batch.device)

        for series in batch:
            out = torch.tensor([]).to(batch.device)
            for image in series:
                out = torch.cat((out, self.features(image.unsqueeze(0))), 0)

            out = self.avg_pool(out).squeeze()
            out = out.max(dim=0, keepdim=True)[0].squeeze()

            out = self.classifier(self.dropout(out))

            batch_out = torch.cat((batch_out, out), 0)

        return batch_out

In [ ]:
def create_losses_csv(out_dir, mri_type):
    loss_path = f'{out_dir}/loss_{mri_type}.csv'

    with open(f'{loss_path}', mode='w') as loss_csv:
        fields = ['train_loss','valid_loss']
        writer = csv.DictWriter(loss_csv, fieldnames=fields)
        writer.writeheader()

    return loss_path

def create_output_dir(exp, mri_type):
    out_dir = f'./{exp}'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    loss_path = create_losses_csv(out_dir,mri_type)

    return out_dir, loss_path

In [ ]:
# data_dir = "../input/mrnet-v1/MRNet-v1.0"
# mri_type = 'T1w'
device = None
lr = 0.00001
weight_decay = 0.01

In [ ]:
def calculate_weights(mri_type, device):

    labels_df = df['MGMT_value']

    neg_count, pos_count = labels_df.value_counts().sort_index()
    weight = torch.tensor([neg_count / pos_count])
    weight = weight.to(device)
    return weight

def make_adam_optimizer(model, lr, weight_decay):
    return optim.Adam(model.parameters(), lr, weight_decay=weight_decay)

def make_lr_scheduler(optimizer,
                      mode='min',
                      factor=0.3,
                      patience=1,
                      verbose=False):
    return optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                mode=mode,
                                                factor=factor,
                                                patience=patience,
                                                verbose=verbose)

In [ ]:
# In order to train ther other models, change mri_type to t1w,flair,or t2w
mri_type = 'T1wCE'
exp = f'{datetime.datetime.now():%Y-%m-%d_%H-%M}'
out_dir, loss_path = create_output_dir(exp, mri_type)
device=None
if device is None:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
train_loader = make_data_loader(directory,mri_type,'train',device=device,shuffle=True)
valid_loader = make_data_loader(directory,mri_type, 'valid', device)

print(f'Creating models...')

# Create a model for each diagnosis

model = MRNet().to(device)

weight = calculate_weights(mri_type, device)
criterion = nn.BCEWithLogitsLoss(pos_weight=weight)

optimizer = make_adam_optimizer(model, lr, weight_decay)

lr_scheduler = make_lr_scheduler(optimizer)

min_valid_loss = np.inf

print(f'Training a model using {mri_type} series...')
print(f'Checkpoints and losses will be save to {out_dir}')

In [ ]:
from sklearn import metrics
# LAbel may be label[0]
def batch_forward_backprop(model, inputs, label, criterion, optimizer):


    model.train()
    optimizer.zero_grad()

    out = model(inputs)
    loss = criterion(out, label)
    loss.backward()
    optimizer.step()

    return loss.item()


def batch_forward(model, inputs, label, criterion):
    
    model.eval()
    out = model(inputs)
    loss = criterion(out, label)
    
    return out.item(), loss.item()


def update_lr_scheduler(lr_scheduler, batch_valid_loss):
    lr_scheduler.step(batch_valid_loss)
        
def calculate_auc(all_labels, all_preds):

    auc = metrics.roc_auc_score(all_labels, all_preds)

    return auc


def print_stats(batch_train_loss, batch_valid_loss,
                valid_label, valid_pred):
    auc = calculate_auc(valid_label, valid_pred)

    print(f'Train losses: {batch_train_loss:.3f},',
          f'\nValid losses: {batch_valid_loss:.3f},',
          f'\nValid AUCs: {auc:.3f},')


def save_losses(train_loss , valid_loss, loss_path):
    with open(f'{loss_path}', mode='a') as loss_csv:
        writer = csv.writer(loss_csv)
        writer.writerow(np.append(train_loss, valid_loss))


def save_checkpoint(epoch, mri_type, model, optimizer, out_dir):
    print('Min valid loss, saving the checkpoint...')

    checkpoint = {
        'epoch': epoch,
        'mri_type': mri_type,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }

    chkpt = f'cnn_{mri_type}_{epoch:2d}.pt'
    torch.save(checkpoint,chkpt)

# Train Model

In [ ]:
epochs=5
for epoch, _ in enumerate(range(epochs), 1):
        print(f'=== Epoch {epoch}/{epochs} ===')

        batch_train_loss = 0.0
        batch_valid_loss = 0.0

        for inputs, labels in tqdm(train_loader,total=len(train_loader)):
            inputs, labels = inputs.to(device), labels.to(device)

            batch_loss = batch_forward_backprop(model, inputs, labels,
                                                criterion, optimizer)
            batch_train_loss += batch_loss
        
        valid_preds = []
        valid_labels = []

        for inputs, labels in tqdm(valid_loader,total=len(valid_loader)):
            inputs, labels = inputs.to(device), labels.to(device)

            batch_preds, batch_loss = batch_forward(model, inputs, labels, criterion)
            batch_valid_loss += batch_loss

            valid_labels.append(labels.detach().cpu().numpy())
            valid_preds.append(batch_preds)

        batch_train_loss /= len(train_loader)
        batch_valid_loss /= len(valid_loader)

        print_stats(batch_train_loss, batch_valid_loss,
                    valid_labels, valid_preds)
        save_losses(batch_train_loss, batch_valid_loss, loss_path)

        update_lr_scheduler(lr_scheduler, batch_valid_loss)

        if batch_valid_loss < min_valid_loss:
            save_checkpoint(epoch, mri_type, model,optimizer, out_dir)

            min_valid_loss = batch_valid_loss

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mri_types = ['FLAIR','T1w','T1wCE','T2w']
models = []
for mri_type in mri_types:
#     checkpoint_pattern = glob.glob(f'./cnn_{mri_type}*.pt')
#     checkpoint_path = sorted(checkpoint_pattern)[-1]
    checkpoint_path = f'../input/models/cnn_{mri_type}.pt'
    checkpoint = torch.load(checkpoint_path, map_location=device)

    model = MRNet().to(device)
    model.load_state_dict(checkpoint['state_dict'])
    models.append(model)

In [ ]:
print(f'Creating data loaders...')

flair_loader = make_data_loader('FLAIR', 'train')
t1w_loader = make_data_loader('T1w', 'train')
t1wce_loader = make_data_loader('T1wCE', 'train')
t2w_loader = make_data_loader('T2w', 'train')

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
print(f'Collecting predictions on train dataset from the models...')

ys= []
Xs = []  # Abnormal, ACL, Meniscus
with tqdm(total=len(flair_loader)) as pbar:
    for (flair_inputs, label), (t1w_inputs, _), (t1wce_inputs, _), (t2w_inputs,_) in \
            zip(flair_loader, t1w_loader, t1wce_loader,t2w_loader):

        flair_inputs, t1w_inputs, t1wce_inputs, t2w_inputs = \
            flair_inputs.to(device),t1w_inputs.to(device), t1wce_inputs.to(device), t2w_inputs.to(device)

        ys.append(label.cpu().tolist())

        flair_prediction = models[0](flair_inputs).detach().cpu().item()
        t1w_prediction = models[1](t1w_inputs).detach().cpu().item()
        t1wce_prediction = models[2](t1wce_inputs).detach().cpu().item()
        t2w_prediction = models[3](t2w_inputs).detach().cpu().item()
        predictions = [flair_prediction,t1w_prediction,t1wce_prediction,t2w_prediction]
        Xs.append(predictions)

        pbar.update(1)




In [ ]:
print(f'Training logistic regression models for each condition...')


clf = LogisticRegressionCV(cv=5, random_state=0).fit(Xs, y)


print(f'Cross validation score: {clf.score(Xs, ys):.3f}')
clf_path = './lr.pkl'
joblib.dump(clf, clf_path)

print(f'Logistic regression models saved to output')

# Submissions

In [ ]:
root_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
submission = pd.read_csv(root_dir+'sample_submission.csv')


In [ ]:
# Load patiennt path
# zfill example, id=5, output = 00005
test_patient_path=[]
for i in range(submission.shape[0]):
    id = submission.iloc[i]["BraTS21ID"]
#     print(str(int(id)))
    test_patient_path.append(os.path.join("../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/",str(int(id)).zfill(5)))

In [ ]:
test_directory ={'FLAIR':[],'T1w':[],'T1wCE':[],'T2w':[]}
for path in test_patient_path:
    test_directory['FLAIR'].append(os.path.join(path, 'FLAIR'))
    test_directory['T1w'].append(os.path.join(path, 'T1w'))
    test_directory['T1wCE'].append(os.path.join(path, 'T1wCE'))
    test_directory['T2w'].append(os.path.join(path, 'T2w'))
    
test_directory = pd.DataFrame(test_directory)

In [ ]:
output_file = 'submission.csv'
mri_types = ['FLAIR','T1w','T1wCE','T2w']
mrnets=[]
for mri_type in mri_types:
    model=MRNet().to(device)
    mrnet_path = f'../input/models/cnn_{mri_type}.pt'
    checkpoint = torch.load(mrnet_path,map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    mrnets.append(model)


In [ ]:
import joblib
lr_model = joblib.load('../input/lr-models/lr.pkl')


transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])

print(f'Generating predictions ')
print(f'Predictions will be saved as {output_file}')


lr_preds=[]
for i in range(len(test_directory)):
    
    data=[]
    for mri_type in mri_types:
        
        path = test_directory[mri_type].iloc[i]
        preprocessed = preprocess_data(path, transform)
        data.append(preprocessed.unsqueeze(0).to(device))
    
    FLAIR_pred = mrnets[0](data[0]).detach().cpu().item()
    T1w_pred = mrnets[1](data[1]).detach().cpu().item()
    T1wCE_pred = mrnets[2](data[2]).detach().cpu().item()
    T2w_pred = mrnets[3](data[3]).detach().cpu().item()
    
    X = [[FLAIR_pred, T1w_pred, T1wCE_pred, T2w_pred]]
    
    lr_preds.append(np.float64(lr_model.predict_proba(X)[:,1]))


In [ ]:
submission['MGMT_value']=lr_preds
submission.to_csv(output_file,index=False)